# Array multinode

In this last notebook we are using several processes to load data in memory but then we are building a single distributed matrix, we can operate on it using methods or even slicing like if it were a `numpy` array, `dask` will take care of managing all the process to process communication for us.

Here we are using `dask-distributed` so we could run on many different nodes.

In [ ]:
def load_file(filename):
    import numpy
    data = numpy.loadtxt(fname=filename, delimiter=',')
    return data

In [ ]:
from glob import glob

In [ ]:
filenames = glob("data/infl*csv")

In [ ]:
# add to the path packages installed with pip install --user
import sys
from os.path import expanduser
sys.path.append(expanduser("~") + "/.local/lib/python3.5/site-packages/")

In [ ]:
from distributed import Executor

In [ ]:
executor = Executor('127.0.0.1:8786')

In [ ]:
data_futures = executor.map(load_file, filenames)

In [ ]:
data_futures

In [ ]:
data_futures[0].result()

In [ ]:
from distributed.collections import futures_to_dask_arrays
distributed_dask_arrays = futures_to_dask_arrays(data_futures)  # many small dask arrays

In [ ]:
import dask.array
all_distributed_data = dask.array.concatenate(distributed_dask_arrays, axis=0)        # one large dask array, joined by time

In [ ]:
all_distributed_data.shape

In [ ]:
all_distributed_data.max().compute()

In [ ]:
executor.compute([all_distributed_data.max(), all_distributed_data[:,1].max()], sync=True)